In [7]:
!pip install -qqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install -qqq --no-deps {xformers} trl peft accelerate bitsandbytes triton --progress-bar off

import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
import wandb

import json
from datasets import Dataset

In [4]:
wandb.login(key = '9abde3fa7ae96f05aa2b8d19c5d7fae16cd0fb74')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/auschra/.netrc


True

## 1. Load model for PEFT

In [5]:
# Load model
max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# Prepare model for PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2080 Ti. Max memory: 10.571 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
Unsloth 2024.11.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605
None


## 2. Prepare data and tokenizer

In [8]:
with open('datasets/ft_nogroup.json', 'r') as f:
    dataset = json.load(f)

tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role" : "from",
             "content" : "value",
             "user" : "human",
             "assistant" : "gpt"}
)

# Function to apply the template
def apply_template(example):
    instruction = example['instruction']
    response = example['response']
    text = f"Instruction: {instruction}\nResponse: {response}"
    return {"text": text}

processed_dataset = [apply_template(example) for example in dataset]
final_dataset = Dataset.from_dict({"text": [item["text"] for item in processed_dataset]})


print(final_dataset[0])

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


{'text': 'Instruction: These are the 16 words of the NYT Connections game: boot, bounce, eject, toss, earth, ground, land, soil, baltic, black, philippine, red, big, philadelphia, splash, sully. What are the four groups and each of their four words?\nResponse: (philadelphia splash sully big) (philippine black red baltic) (ground earth soil land) (boot bounce toss eject)'}


## 3. Training

In [9]:
# init wandb
# Initialize W&B project
wandb.init(project="Connections-nogroups-sets", config={
    "learning_rate": 3e-4,
    "batch_size": 4,
    "epochs": 1,
    "optimizer": "adamw_8bit"
})

# Modify TrainingArguments to report to W&B
training_args = TrainingArguments(
    learning_rate=3e-4,
    lr_scheduler_type="linear",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    warmup_steps=10,
    output_dir="output",
    seed=0,
    report_to="wandb",  # Enable W&B logging
)


max_seq_length = 512
# Trainer setup
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=final_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=True,
    args=training_args,
)

trainer.train()

wandb: Currently logged in as: auschra3 (auschra3-massachusetts-institute-of-technology). Use `wandb login --relogin` to force relogin


Generating train split: 942 examples [00:00, 2028.00 examples/s]
/home/auschra/ml/connectionsbot/venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 942 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 59
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  2%|▏         | 1/59 [00:12<12:22, 12.81s/it]

{'loss': 2.1947, 'grad_norm': 3.1580238342285156, 'learning_rate': 2.9999999999999997e-05, 'epoch': 0.02}


  3%|▎         | 2/59 [00:24<11:25, 12.02s/it]

{'loss': 2.203, 'grad_norm': 2.9665374755859375, 'learning_rate': 5.9999999999999995e-05, 'epoch': 0.03}


...

100%|██████████| 59/59 [12:02<00:00, 11.74s/it]

{'loss': 0.627, 'grad_norm': 1.3656067848205566, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 59/59 [12:04<00:00, 12.27s/it]

{'train_runtime': 724.1585, 'train_samples_per_second': 1.301, 'train_steps_per_second': 0.081, 'train_loss': 1.1121663986626318, 'epoch': 1.0}


TrainOutput(global_step=59, training_loss=1.1121663986626318, metrics={'train_runtime': 724.1585, 'train_samples_per_second': 1.301, 'train_steps_per_second': 0.081, 'total_flos': 2.177861902191821e+16, 'train_loss': 1.1121663986626318, 'epoch': 1.0})

## 4. Inference

In [27]:
# prompt
#These are the 16 words of the NYT Connections game: {words} What are the four groups and each of their four words?

# Load model for inference
model = FastLanguageModel.for_inference(model)

# Define the words
words = 'stop, poem, different, message, new, novel, yardstick, furniture, play, text, biped, record, correspond, original, pause, write'

# Create a prompt that includes only the question (without the instruction)
prompt = f"These are the 16 words of the NYT Connections game: {words} What are the four groups and each of their four words?"

instruction = [
    {
        "role": "human",
        "content": prompt,
    },
]
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Stream the output from the model, limiting it to just the answer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs["input_ids"], streamer=text_streamer, max_new_tokens=128, temperature=0.7, top_p=0.9, use_cache=True)



<|begin_of_text|>These are the 16 words of the NYT Connections game: stop, poem, different, message, new, novel, yardstick, furniture, play, text, biped, record, correspond, original, pause, write What are the four groups and each of their four words? Response: (yardstick novel furniture new) (pause correspond original write) (text biped record play) (poem stop message different)<|im_end|>


## 5. Save trained model

In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
model.push_to_hub_merged("mlabonne/FineLlama-3.1-8B", tokenizer, save_method="merged_16bit")

In [ ]:
model.save_pretrained_gguf("model", tokenizer, "q8_0")
quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
for quant in quant_methods:
    model.push_to_hub_gguf("mlabonne/FineLlama-3.1-8B-GGUF", tokenizer, quant)